PROYECTO INTELIGENCIA COMPUTACIONAL:

- Maximiliano Quintero
- Tomás Apablaza

# 1) Librerías

In [1]:


# pyarrow might be needed to read the data
!python -m pip install Cython
!python -m pip install -e git+https://git@github.com/alercebroker/turbo-fats#egg=turbofats
!python -m pip install -e git+https://git@github.com/alercebroker/mhps#egg=mhps
!python -m pip install -e git+https://git@github.com/alercebroker/P4J#egg=P4J
!python -m pip install pyarrow
!python -m pip install -e git+https://git@github.com/alercebroker/lc_classifier#egg=lc_classifier

Obtaining turbofats from git+https://****@github.com/alercebroker/turbo-fats#egg=turbofats
  Cloning https://****@github.com/alercebroker/turbo-fats to ./src/turbofats
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/alercebroker/turbo-fats' /content/src/turbofats
  Resolved https://****@github.com/alercebroker/turbo-fats to commit c9b67f4087df35fe4e85c3e09e3a8e1be0e31068
  Preparing metadata (setup.py) ... done
  Running setup.py develop for turbofats
Obtaining mhps from git+https://****@github.com/alercebroker/mhps#egg=mhps
  Cloning https://****@github.com/alercebroker/mhps to ./src/mhps
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/alercebroker/mhps' /content/src/mhps
  Resolved https://****@github.com/alercebroker/mhps to commit 1c827b7d19a44ecbd54d79dd2cd786ed17c4ce91
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... don

Reiniciar entorno de ejecución y partir desde acá:

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from lc_classifier.utils import LightcurveBuilder
from lc_classifier.features import MHPSExtractor, PeriodExtractor, GPDRWExtractor
from lc_classifier.features import FoldedKimExtractor
from lc_classifier.features import HarmonicsExtractor, IQRExtractor
from lc_classifier.features import PowerRateExtractor
from lc_classifier.features import TurboFatsFeatureExtractor
from lc_classifier.features import FeatureExtractorComposer

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 3) Feature extractor

# REINICIA ENTORNO DE EJECUCIÓN PARA ELIMINAR ERROR

In [4]:
bands = ["r","u"]

feature_extractor = FeatureExtractorComposer(
    [
        MHPSExtractor(bands),
        PeriodExtractor(bands, smallest_period = 1/1000, largest_period = 1/20, optimal_grid = False, min_length = 5., trim_lightcurve_to_n_days= 10000),
        GPDRWExtractor(bands),
        FoldedKimExtractor(bands),
        HarmonicsExtractor(bands),
        IQRExtractor(bands),
        PowerRateExtractor(bands),
        TurboFatsFeatureExtractor(bands)
    ]
)

In [ ]:
import os
import pandas as pd
import gdown

ruta_bases =  '/content/drive/MyDrive/Base de datos IC/'



# Bucle para cargar y añadir los DataFrames a la lista
for i in range(1019, 1119):
    base_name = f"base_estrellas_{i}"

    # Construir la ruta completa de la base de datos
    base_path = os.path.join(ruta_bases, f"{base_name}.csv")

    # Leer la base de datos
    base_df = pd.read_csv(base_path)

    # Filtrar base
    base_df = base_df[["oid","time","magnitude","error","band"]]

    # Setear indice oid
    base_df = base_df.set_index('oid')

    # Definir las características del feature extractor
    bands = ["r","u"]

    feature_extractor = FeatureExtractorComposer(
        [
            MHPSExtractor(bands),
            PeriodExtractor(bands, smallest_period = 1/1000, largest_period = 1/20, optimal_grid = False, min_length = 5., trim_lightcurve_to_n_days= 10000),
            GPDRWExtractor(bands),
            FoldedKimExtractor(bands),
            HarmonicsExtractor(bands),
            IQRExtractor(bands),
            PowerRateExtractor(bands),
            TurboFatsFeatureExtractor(bands)
        ]
    )

    # Aplicar la función de extracción de características
    result = feature_extractor.compute_features(base_df)

    # Guardar la base de datos resultante en Google Drive
    id_carpeta_drive = f"/content/drive/MyDrive/Base de datos IC/Resultados/base_resultante_{i}.csv"
    result.to_csv(id_carpeta_drive, index=True)
    print("Base_",i," lista")
